# Keras

* `tf.contrib.keras` 를 쓰니 좀 이런저런 문제가 있어서 따로 인스톨해서 사용함

* keras 를 이용해서 tutorial 을 구현해보자.
* keras 를 이용해서 transfer learning 을 구현해보자.
* 추가 - slim 을 이용해서 transfer learning 을 구현해보자.

## 참고

텐서보드 등으로 케라스를 모니터링 하고 싶다면, https://tykimos.github.io/Keras/2017/07/09/Training_Monitoring/ 를 참고하자 (한글자료).

## TF Tutorial using Keras

* BN model

In [1]:
# from keras.datasets import mnist
from tensorflow.examples.tutorials.mnist import input_data
import keras
from keras import backend as K
from keras.layers import Conv2D, BatchNormalization, Activation, MaxPooling2D, Dense, Flatten, Dropout
import numpy as np

Using TensorFlow backend.


In [2]:
keras.__version__

'2.0.6'

In [3]:
# GPU memory control in keras
# import tensorflow as tf
# config = tf.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.4
# K.tensorflow_backend.set_session(tf.Session(config=config))

In [4]:
np.random.seed(0)

In [5]:
# (x_tv, y_tv), (x_test, y_test) = mnist.load_data()
# x_train, x_valid = x_tv[:55000], x_tv[55000:]
# y_train, y_valid = y_tv[:55000], y_tv[55000:]

In [6]:
mnist = input_data.read_data_sets('./MNIST_data', one_hot=True)

Extracting ./MNIST_data/train-images-idx3-ubyte.gz
Extracting ./MNIST_data/train-labels-idx1-ubyte.gz
Extracting ./MNIST_data/t10k-images-idx3-ubyte.gz
Extracting ./MNIST_data/t10k-labels-idx1-ubyte.gz


In [7]:
# Keras 에서도 mnist 를 제공하지만 데이터셋을 동일하게 맞추기 위해 tf 의 mnist 를 사용하자.
X_train, y_train = mnist.train.images, mnist.train.labels
X_valid, y_valid = mnist.validation.images, mnist.validation.labels
X_test, y_test = mnist.test.images, mnist.test.labels

In [8]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_valid = X_valid.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

## Sequential API

In [9]:
# Sequential model
from keras.models import Sequential

In [10]:
def build_model_sequential(input_shape, output_units):
    model = Sequential()

    n_filters = 32
    for i in range(3):
        if i == 0:
            model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same', input_shape=input_shape))
        else:
            model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(Conv2D(n_filters, [3,3], use_bias=False, padding='same'))
        model.add(BatchNormalization())
        model.add(Activation('relu'))
        model.add(MaxPooling2D())
        model.add(Dropout(0.3))
        
        n_filters *= 2

    model.add(Flatten())
    model.add(Dense(output_units, activation='softmax'))

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [11]:
model = build_model_sequential([28, 28, 1], 10)

In [12]:
result = model.fit(X_train, y_train, batch_size=128, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 55000 samples, validate on 10000 samples
Epoch 1/100
55000/55000 [==============================] - 9s - loss: 0.2252 - acc: 0.9306 - val_loss: 0.1280 - val_acc: 0.9589
Epoch 2/100
55000/55000 [==============================] - 8s - loss: 0.0610 - acc: 0.9807 - val_loss: 0.0442 - val_acc: 0.9864
Epoch 3/100
55000/55000 [==============================] - 8s - loss: 0.0456 - acc: 0.9858 - val_loss: 0.0245 - val_acc: 0.9924
Epoch 4/100
55000/55000 [==============================] - 8s - loss: 0.0381 - acc: 0.9879 - val_loss: 0.0237 - val_acc: 0.9919
Epoch 5/100
55000/55000 [==============================] - 8s - loss: 0.0353 - acc: 0.9884 - val_loss: 0.0204 - val_acc: 0.9930
Epoch 6/100
55000/55000 [==============================] - 8s - loss: 0.0290 - acc: 0.9909 - val_loss: 0.0167 - val_acc: 0.9940
Epoch 7/100
55000/55000 [==============================] - 8s - loss: 0.0255 - acc: 0.9919 - val_loss: 0.0326 - val_acc: 0.9896
Epoch 8/100
55000/55000 [=============================

In [13]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

99.60%


## Functional API

시퀀셜 모델과 동일하나 펑셔널 API 가 더 flexible 한 듯 함.

In [14]:
from keras.layers import Input
from keras.models import Model

In [15]:
def build_model_functional(input_shape, output_units):
    input_tensor = Input(input_shape)
    net = input_tensor
    n_filters = 32

    for _ in range(3):
        net = Conv2D(n_filters, [3,3], padding='same')(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = Conv2D(n_filters, [3,3], padding='same')(net)
        net = BatchNormalization()(net)
        net = Activation('relu')(net)
        net = MaxPooling2D()(net)
        net = Dropout(0.3)(net)
        
        n_filters *= 2

    net = Flatten()(net)
    net = Dense(output_units, activation='softmax')(net)

    model = Model(input_tensor, net)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return model

In [16]:
K.clear_session()

model = build_model_functional([28, 28, 1], 10)

In [17]:
result = model.fit(X_train, y_train, batch_size=128, epochs=100, verbose=1, validation_data=(X_test, y_test))

Train on 55000 samples, validate on 10000 samples
Epoch 1/100
55000/55000 [==============================] - 9s - loss: 0.2471 - acc: 0.9247 - val_loss: 0.1693 - val_acc: 0.9467
Epoch 2/100
55000/55000 [==============================] - 9s - loss: 0.0641 - acc: 0.9794 - val_loss: 0.0350 - val_acc: 0.9880
Epoch 3/100
55000/55000 [==============================] - 9s - loss: 0.0451 - acc: 0.9853 - val_loss: 0.0248 - val_acc: 0.9914
Epoch 4/100
55000/55000 [==============================] - 9s - loss: 0.0392 - acc: 0.9877 - val_loss: 0.0228 - val_acc: 0.9928
Epoch 5/100
55000/55000 [==============================] - 9s - loss: 0.0326 - acc: 0.9897 - val_loss: 0.0203 - val_acc: 0.9931
Epoch 6/100
55000/55000 [==============================] - 9s - loss: 0.0294 - acc: 0.9904 - val_loss: 0.0277 - val_acc: 0.9907
Epoch 7/100
55000/55000 [==============================] - 9s - loss: 0.0265 - acc: 0.9915 - val_loss: 0.0259 - val_acc: 0.9909
Epoch 8/100
55000/55000 [=============================

In [18]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

99.54%


## Data augmentation

In [21]:
from keras.preprocessing.image import ImageDataGenerator

In [23]:
K.clear_session()

model = build_model_functional([28, 28, 1], 10)

In [24]:
datagen = ImageDataGenerator(rotation_range=15, width_shift_range=0.1, height_shift_range=0.1, zoom_range=0.1)
datagen.fit(X_train)

/home/nhnent/tf-latest-env/local/lib/python2.7/site-packages/keras/preprocessing/image.py:653: UserWarning: Expected input to be images (as Numpy array) following the data format convention "channels_last" (channels on axis 3), i.e. expected either 1, 3 or 4 channels on axis 3. However, it was passed an array with shape (55000, 28, 28, 1) (1 channels).
  ' (' + str(x.shape[self.channel_axis]) + ' channels).')


In [25]:
result = model.fit_generator(datagen.flow(X_train, y_train, batch_size=128), steps_per_epoch=X_train.shape[0] // 128,
                             epochs=100, validation_data=(X_test, y_test))

Epoch 1/100
429/429 [==============================] - 8s - loss: 0.4286 - acc: 0.8643 - val_loss: 0.0583 - val_acc: 0.9848
Epoch 2/100
429/429 [==============================] - 8s - loss: 0.1164 - acc: 0.9639 - val_loss: 0.0471 - val_acc: 0.9860
Epoch 3/100
429/429 [==============================] - 8s - loss: 0.0839 - acc: 0.9739 - val_loss: 0.0271 - val_acc: 0.9902
Epoch 4/100
429/429 [==============================] - 8s - loss: 0.0733 - acc: 0.9774 - val_loss: 0.0227 - val_acc: 0.9922
Epoch 5/100
429/429 [==============================] - 8s - loss: 0.0611 - acc: 0.9810 - val_loss: 0.0195 - val_acc: 0.9941
Epoch 6/100
429/429 [==============================] - 8s - loss: 0.0578 - acc: 0.9821 - val_loss: 0.0212 - val_acc: 0.9937
Epoch 7/100
429/429 [==============================] - 8s - loss: 0.0534 - acc: 0.9835 - val_loss: 0.0205 - val_acc: 0.9932
Epoch 8/100
429/429 [==============================] - 8s - loss: 0.0493 - acc: 0.9843 - val_loss: 0.0195 - val_acc: 0.9931
Epoch 9/

In [26]:
print "{:.2%}".format(np.average(result.history['val_acc'][-5:]))

99.62%
